# Product Recommendation
Reference: https://ieeexplore.ieee.org/document/5430993

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

In [2]:
tf.__version__

'2.3.1'

## Data Preprocessing

In [3]:
Y_data = pd.read_csv('data/Y.csv', header=None, names=['Rating','Movie','User'], dtype=np.int32) # training data
P_data = pd.read_csv('data/P.csv', header=None, names=['Rating','Movie','User'], dtype=np.int32) # test data ('probe-set' mentioned in paper)

In [4]:
display(Y_data.head())
display(P_data.head())
Y_data.shape, P_data.shape

,Rating,Movie,User
0,5,2,1
1,4,7,1
2,4,8,1
3,4,11,1
4,4,12,1


,Rating,Movie,User
0,3,6,1
1,5,96,1
2,3,1,2
3,3,33,3
4,5,42,4


((3399874, 3), (189699, 3))

In [5]:
print(Y_data['Rating'].unique().max(), Y_data['Movie'].unique().max(), Y_data['User'].unique().max())
print(P_data['Rating'].unique().max(), P_data['Movie'].unique().max(), P_data['User'].unique().max())

5 100 137328
5 100 137328


In [6]:
k, n = Y_data['Movie'].unique().max(), Y_data['User'].unique().max()
k, n

(100, 137328)

In [7]:
def generate_indices_pair_list(data):
    user_id = 1
    indices_list = list()
    for index, row in tqdm(data.iterrows(), total=data.shape[0]):
        if row['User'] != user_id:
            user_id = row['User']
            indices_list.append(index - 1)

    indices_pair_list = list()
    for index_ending in indices_list:
        if index_ending == indices_list[0]:
            indices_pair_list.append((0, index_ending))
        else:
            index_beginning = indices_pair_list[-1][1] + 1
            indices_pair_list.append((index_beginning, index_ending))
    return indices_pair_list

In [8]:
indices_pair_list_Y_data = generate_indices_pair_list(Y_data)
indices_pair_list_P_data = generate_indices_pair_list(P_data)
len(indices_pair_list_Y_data) == len(indices_pair_list_P_data)

100%|██████████| 189699/189699 [00:13<00:00, 14002.31it/s]


True

In [9]:
data_preprocessed = list()
for index_pair_Y_data, index_pair_P_data in tqdm(zip(indices_pair_list_Y_data, indices_pair_list_P_data), total=len(indices_pair_list_Y_data)):
    Y_data_t = Y_data.loc[index_pair_Y_data[0]:index_pair_Y_data[1], :]
    
    movie_ids_t_indices = Y_data_t['Movie'].values - 1    
    H_yt = tf.constant(np.identity(k)[movie_ids_t_indices], dtype=tf.float32)
    H_xt = tf.constant(np.delete(np.identity(k), movie_ids_t_indices, 0), dtype=tf.float32)    
    k_t = tf.constant(H_yt.shape[0], dtype=tf.float32)
    
    z_t_st_indices = np.vstack((movie_ids_t_indices, np.zeros(movie_ids_t_indices.shape[0], dtype=np.int64))).T
    z_t = tf.sparse.to_dense(tf.SparseTensor(indices=z_t_st_indices, values=Y_data_t['Rating'].values.astype(np.float32), dense_shape=[k, 1]))
    
    y_t = tf.matmul(H_yt, z_t)
    x_t = tf.matmul(H_xt, z_t)
    
    P_data_t = P_data.loc[index_pair_P_data[0]:index_pair_P_data[1], :]
    movie_ids_t_P_data = P_data_t['Movie'].values
    ratings_t_P_data = tf.expand_dims(P_data_t['Rating'].values.astype(np.float32), axis=1)
    data_preprocessed.append((H_yt, H_xt, tf.transpose(H_yt), tf.transpose(H_xt), k_t, z_t, y_t, x_t, movie_ids_t_P_data, ratings_t_P_data))
        
del Y_data
del P_data

100%|██████████| 137327/137327 [02:48<00:00, 815.65it/s] 


## Initialization
$\mu$ has 1 type available <br />
$N = \sum_{t=1}^{n}H_{y_t}'H_{y_t}$ <br />

$\hat{\mu}^0 = N^{-1}\sum_{t-1}^{n}H_{y_t}'y_{t}$

In [10]:
# initial estimate of mu
N = 0
H_yty_t = 0
    
for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
    N += tf.matmul(H_yt_trans, H_yt)
    H_yty_t += tf.matmul(H_yt_trans, y_t)

100%|██████████| 137327/137327 [00:10<00:00, 13569.33it/s]


In [11]:
# The ith diagonal element of N equals the total number of ratings of the ith product.
N

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[20017.,     0.,     0., ...,     0.,     0.,     0.],
       [    0., 23916.,     0., ...,     0.,     0.,     0.],
       [    0.,     0., 31634., ...,     0.,     0.,     0.],
       ...,
       [    0.,     0.,     0., ..., 60895.,     0.,     0.],
       [    0.,     0.,     0., ...,     0., 61520.,     0.],
       [    0.,     0.,     0., ...,     0.,     0., 64505.]],
      dtype=float32)>

In [12]:
mu_hat0 = tf.matmul(tf.linalg.inv(N), H_yty_t)
tf.transpose(mu_hat0)

<tf.Tensor: shape=(1, 100), dtype=float32, numpy=
array([[3.4526653, 3.5767686, 3.2878866, 3.9047875, 3.7903547, 3.4441562,
        3.190684 , 4.5283504, 3.8201292, 3.6159503, 3.4038272, 3.8372512,
        4.076039 , 4.228367 , 3.3539274, 4.0645275, 3.7211962, 3.4870086,
        4.1638894, 3.4097998, 3.86926  , 3.435835 , 3.2032225, 4.084879 ,
        3.2320046, 3.886682 , 4.331895 , 4.383559 , 4.316363 , 3.8659174,
        4.339757 , 3.8914747, 3.7002807, 3.3624778, 4.3289886, 4.0670323,
        4.56922  , 3.771041 , 3.6858559, 3.8453238, 4.345388 , 3.9099529,
        3.3994992, 3.6078188, 3.9626696, 4.143861 , 3.4072049, 3.7040153,
        4.0034695, 4.6428022, 3.216206 , 3.7723858, 4.265651 , 4.4537544,
        3.8384895, 3.793742 , 3.76288  , 3.8869822, 3.8004174, 4.346952 ,
        3.8046887, 3.8462136, 3.6412156, 3.2722168, 3.4232938, 3.7163155,
        3.2069893, 4.4541044, 4.265392 , 3.8610888, 4.483009 , 4.3621464,
        3.5388138, 4.1171074, 3.8946686, 3.3607738, 4.179405 ,

R has 4 types available <br />
$R_{1} = I$ <br />

$R_{2} = N^{-1}diag(S)$ <br />

$R_{3} = diag(S)^{-1/2}Sdiag(S)^{-1/2}$ <br />

$R_{4} = N^{-1/2}SN^{-1/2}$ <br />

where $S = \sum_{t=1}^{n}H_{y_{t}}'(y_t - H_{y_{t}}\hat{\mu}^0)(y_t - H_{y_{t}}\hat{\mu}^0)'H_{y_{t}}$

In [13]:
# initial estimates of R (4 types available)
R_hat0_1 = tf.eye(k, dtype=tf.float32)
R_hat0_1

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [14]:
S = 0
for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
    Hytmu_hat0 = tf.matmul(H_yt, mu_hat0)
    S += H_yt_trans @ (y_t - Hytmu_hat0) @ tf.transpose(y_t - Hytmu_hat0) @ H_yt

100%|██████████| 137327/137327 [00:20<00:00, 6749.16it/s]


In [15]:
# diag_S is the diagonal matrix consisting of the diagonal elements of S
diag_S = tf.linalg.diag(tf.linalg.tensor_diag_part(S))
R_hat0_2 = tf.matmul(tf.linalg.inv(N), diag_S)
R_hat0_2

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[1.724364  , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.94218737, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.4365153 , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.1832287 , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.0349729 ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.2620498 ]], dtype=float32)>

In [16]:
# R_hat0_3 is not a good initializer when rating variances are far from one
R_hat0_3 = tf.matmul(tf.linalg.sqrtm(tf.linalg.inv(diag_S)), tf.matmul(S, tf.linalg.sqrtm(tf.linalg.inv(diag_S))))
R_hat0_3

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[ 1.        ,  0.07418733, -0.01158332, ..., -0.01462825,
        -0.02215315, -0.0184503 ],
       [ 0.07418733,  0.99999994,  0.03674516, ...,  0.02561875,
         0.03563373,  0.0392758 ],
       [-0.01158332,  0.03674517,  0.9999999 , ...,  0.1095439 ,
         0.12823555,  0.15562423],
       ...,
       [-0.01462825,  0.02561875,  0.1095439 , ...,  1.0000001 ,
         0.1978014 ,  0.15164192],
       [-0.02215315,  0.03563373,  0.12823555, ...,  0.19780138,
         1.0000001 ,  0.18996929],
       [-0.0184503 ,  0.0392758 ,  0.15562423, ...,  0.15164192,
         0.18996929,  1.0000001 ]], dtype=float32)>

In [17]:
R_hat0_4 = tf.matmul(tf.linalg.sqrtm(tf.linalg.inv(N)), tf.matmul(S, tf.linalg.sqrtm(tf.linalg.inv(N))))
R_hat0_4

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[ 1.724364  ,  0.09456117, -0.01823068, ..., -0.02089494,
        -0.02959474, -0.02721799],
       [ 0.09456117,  0.94218737,  0.04274881, ...,  0.0270496 ,
         0.03518798,  0.04282841],
       [-0.01823068,  0.04274881,  1.4365153 , ...,  0.14281628,
         0.15636086,  0.20954175],
       ...,
       [-0.02089494,  0.0270496 ,  0.14281628, ...,  1.1832289 ,
         0.21889113,  0.18530701],
       [-0.02959474,  0.03518798,  0.15636086, ...,  0.21889113,
         1.0349729 ,  0.21711312],
       [-0.02721799,  0.0428284 ,  0.20954177, ...,  0.18530701,
         0.21711312,  1.2620498 ]], dtype=float32)>

## Expectation Maximization Algorithm

$\hat{\mu} = \big(\sum_{t=1}^{n}H_{y_t}'R_{y_t}^{-1}H_{y_t}\big)^{-1} \big(\sum_{t=1}^{n}H_{y_t}'R_{y_t}^{-1}y_t\big)$

$\hat{R}^{i+1} = \frac{1}{n} \sum_{t=1}^{n} (\hat{Z}_t - \mu)(\hat{Z}_t - \mu)' + H_{x_t}' \bigg(\hat{R}_{x_t}^{i} - \hat{R}_{x_ty_t}^{i} \big(\hat{R}_{y_t}^{i}\big)^{-1} \big(\hat{R}_{x_ty_t}^{i}\big)'\bigg) H_{x_t}$

In [18]:
LOG_2PI = tf.math.log(2*tf.constant(np.pi, dtype=tf.float32))

@tf.function(experimental_relax_shapes=True)
def run_graph_em(mu, R, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans, k_t):
    # for R estimation
    R_xt = H_xt @ R @ H_xt_trans
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_det = tf.linalg.det(R_yt)
    R_yt_inv = tf.linalg.inv(R_yt)
    R_xtyt = H_xt @ R @ H_yt_trans

    mu_yt = tf.matmul(H_yt, mu)
    mu_xt = tf.matmul(H_xt, mu)

    X_t_hat = R_xtyt @ R_yt_inv @ (y_t - mu_yt) + mu_xt
    Z_t_hat = H_yt_trans @ y_t + H_xt_trans @ X_t_hat
    
    R_hat_sum_part = (Z_t_hat - mu) @ tf.transpose(Z_t_hat - mu) \
                        + H_xt_trans @ (R_xt - R_xtyt @ R_yt_inv @ tf.transpose(R_xtyt)) @ H_xt

    # for mu estimation
    Hyt_trans_Ryt_inv_Hyt_sum_part = H_yt_trans @ R_yt_inv @ H_yt
    Hyt_trans_Ryt_inv_yt_sum_part = H_yt_trans @ R_yt_inv @ y_t
    
    # for log likelihood calculation
    log_p_hat_part = -1/2*(tf.math.log(R_yt_det) + tf.transpose(y_t - mu_yt) @ R_yt_inv @ (y_t - mu_yt) + k_t*LOG_2PI) 
    
    return R_hat_sum_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part

In [19]:
def expectation_maximization(mu, R):
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    R_hat_sum = 0
    log_p_hat = 0
    
    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
        R_hat_sum_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part = \
            run_graph_em(mu, R, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans, k_t)
        
        R_hat_sum += R_hat_sum_part
        Hyt_trans_Ryt_inv_Hyt_sum += Hyt_trans_Ryt_inv_Hyt_sum_part
        Hyt_trans_Ryt_inv_yt_sum += Hyt_trans_Ryt_inv_yt_sum_part
        log_p_hat += log_p_hat_part
        
    R_hat = R_hat_sum / n
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)    
    return mu_hat, R_hat, log_p_hat

In [20]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = tf.constant(-np.inf, dtype=tf.float32)

for i in range(30):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = expectation_maximization(mu, R)
    convergence_criterion = log_p_hat/n - log_p/n < delta
    
    print(f'normalized log_p_hat: {(log_p_hat/n).numpy().flatten()[0]:.5}')
    print(f'normalized log_p:     {(log_p/n).numpy().flatten()[0]:.5}')
    print(f'convergence gap:      {(log_p_hat/n - log_p/n).numpy().flatten()[0]:.5}')
    
    if convergence_criterion:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 0/137327 [00:00<?, ?it/s]

iteration: 0


  0%|          | 169/137327 [00:00<01:21, 1684.57it/s]

normalized log_p_hat: -32.232
normalized log_p:     -inf
convergence gap:      inf


  0%|          | 126/137327 [00:00<01:48, 1259.32it/s]

...

normalized log_p_hat: -31.414
normalized log_p:     -31.414
convergence gap:      0.00056076
iteration: 25


  0%|          | 138/137327 [00:00<01:39, 1372.12it/s]

normalized log_p_hat: -31.413
normalized log_p:     -31.414
convergence gap:      0.00052834


100%|██████████| 137327/137327 [01:27<00:00, 1575.96it/s]

normalized log_p_hat: -31.413
normalized log_p:     -31.413
convergence gap:      0.00042915


In [21]:
# 26 iterations, ~38 min
np.save('results/em_mu.npy', mu_hat)
np.save('results/em_R.npy', R_hat)
np.save('results/em_log_p.npy', log_p_hat)

## McMichael’s Algorithm

$\hat{\mu} = \big(\sum_{t=1}^{n}H_{y_t}'R_{y_t}^{-1}H_{y_t}\big)^{-1} \big(\sum_{t=1}^{n}H_{y_t}'R_{y_t}^{-1}y_t\big)$

$\hat{R}^{i+1} = \hat{R}^{i} + \gamma \hat{R}^{i} \bigg(\frac{d}{dR} \log{p}(y^n; \mu, R)|_{R = \hat{R}^i} \bigg) \hat{R}^{i}$

where $\frac{d}{dR} \log{p}(y^n; \mu, R)|_{R = \hat{R}^i} = -\frac{1}{2} \sum_{t=1}^{n} H_{y_t}' \Big(\big(R_{y_t}^{i}\big)^{-1} - \big(R_{y_t}^{i}\big)^{-1} (y_t - \mu_{y_t}) (y_t - \mu_{y_t})' \big(R_{y_t}^{i}\big)^{-1}\Big) H_{y_t}$

In [22]:
@tf.function(experimental_relax_shapes=True)
def run_graph_mcmichael(mu, R, y_t, H_yt, H_yt_trans, k_t):
    # for R estimation
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_det = tf.linalg.det(R_yt)
    R_yt_inv = tf.linalg.inv(R_yt)
    mu_yt = tf.matmul(H_yt, mu)
    log_p_gradient_part = H_yt_trans @ (R_yt_inv - R_yt_inv @ (y_t - mu_yt) @ tf.transpose(y_t - mu_yt) @ R_yt_inv) @ H_yt

    # for mu estimation
    Hyt_trans_Ryt_inv_Hyt_sum_part = H_yt_trans @ R_yt_inv @ H_yt
    Hyt_trans_Ryt_inv_yt_sum_part = H_yt_trans @ R_yt_inv @ y_t
    
    # for log likelihood calculation
    log_p_hat_part = -1/2*(tf.math.log(R_yt_det) + tf.transpose(y_t - mu_yt) @ R_yt_inv @ (y_t - mu_yt) + k_t*LOG_2PI)
    
    return log_p_gradient_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part

In [23]:
def mcmichael(mu, R):
    gamma = 0.00001
    Hyt_trans_Ryt_inv_Hyt_sum = 0
    Hyt_trans_Ryt_inv_yt_sum = 0
    log_p_gradient = 0
    log_p_hat = 0

    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
        log_p_gradient_part, Hyt_trans_Ryt_inv_Hyt_sum_part, Hyt_trans_Ryt_inv_yt_sum_part, log_p_hat_part = \
            run_graph_mcmichael(mu, R, y_t, H_yt, H_yt_trans, k_t)
        
        log_p_gradient += log_p_gradient_part
        Hyt_trans_Ryt_inv_Hyt_sum += Hyt_trans_Ryt_inv_Hyt_sum_part
        Hyt_trans_Ryt_inv_yt_sum += Hyt_trans_Ryt_inv_yt_sum_part
        log_p_hat += log_p_hat_part
        
    R_hat = R + gamma*(R @ (-1/2*log_p_gradient) @ R)
    mu_hat = tf.matmul(tf.linalg.inv(Hyt_trans_Ryt_inv_Hyt_sum), Hyt_trans_Ryt_inv_yt_sum)
    return mu_hat, R_hat, log_p_hat

In [24]:
delta = 0.0005
mu = mu_hat0
R = R_hat0_4
log_p = tf.constant(-np.inf, dtype=tf.float32)

for i in range(40):
    if i % 5 == 0:
        print(f'iteration: {i}')
    
    mu_hat, R_hat, log_p_hat = mcmichael(mu, R)
    convergence_criterion = log_p_hat/n - log_p/n < delta
    
    print(f'normalized log_p_hat: {(log_p_hat/n).numpy().flatten()[0]:.5}')
    print(f'normalized log_p:     {(log_p/n).numpy().flatten()[0]:.5}')
    print(f'convergence gap:      {(log_p_hat/n - log_p/n).numpy().flatten()[0]:.5}')
    
    if convergence_criterion:
        break
        
    # use new estimattions for next iteration
    mu = mu_hat
    R = R_hat
    log_p = log_p_hat

  0%|          | 1/137327 [00:00<5:44:25,  6.65it/s]

iteration: 0


  0%|          | 366/137327 [00:00<01:19, 1729.54it/s]

normalized log_p_hat: -32.232
normalized log_p:     -inf
convergence gap:      inf


  0%|          | 198/137327 [00:00<01:09, 1974.90it/s]

...

normalized log_p_hat: -31.415
normalized log_p:     -31.416
convergence gap:      0.00063515


  0%|          | 402/137327 [00:00<01:07, 2015.75it/s]

normalized log_p_hat: -31.415
normalized log_p:     -31.415
convergence gap:      0.00051689


100%|██████████| 137327/137327 [01:08<00:00, 1999.55it/s]

normalized log_p_hat: -31.414
normalized log_p:     -31.415
convergence gap:      0.00044441


In [25]:
# 35 iterations, ~38 min
np.save('results/mcmichael_mu.npy', mu_hat)
np.save('results/mcmichael_R.npy', R_hat)
np.save('results/mcmichael_log_p.npy', log_p_hat)

## Evaluation

In [26]:
@tf.function(experimental_relax_shapes=True)
def run_graph_square_error(mu, R, movie_ids_t_P_data, ratings_t_P_data, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans):
    # calculate X_t_hat
    R_xt = H_xt @ R @ H_xt_trans
    R_yt = H_yt @ R @ H_yt_trans
    R_yt_inv = tf.linalg.inv(R_yt)
    R_xtyt = H_xt @ R @ H_yt_trans

    mu_yt = tf.matmul(H_yt, mu)
    mu_xt = tf.matmul(H_xt, mu)

    X_t_hat = R_xtyt @ R_yt_inv @ (y_t - mu_yt) + mu_xt
    
    # clip ratings
    predictions_t = tf.gather(tf.matmul(H_xt_trans, X_t_hat), indices=movie_ids_t_P_data-1)
    predictions_t = tf.clip_by_value(predictions_t, 1, 5)
    
    return tf.matmul(tf.transpose(ratings_t_P_data - predictions_t), ratings_t_P_data - predictions_t)

In [29]:
def evaluate(mu, R):
    square_error = 0
    l = 0
    for (H_yt, H_xt, H_yt_trans, H_xt_trans, k_t, Z_t, y_t, x_t, movie_ids_t_P_data, ratings_t_P_data) in tqdm(data_preprocessed):
        square_error += run_graph_square_error(mu, R, movie_ids_t_P_data, ratings_t_P_data, y_t, H_xt, H_xt_trans, H_yt, H_yt_trans)
        l += len(ratings_t_P_data)
    return np.sqrt(square_error/l)

In [30]:
em_mu = np.load('results/em_mu.npy')
em_R = np.load('results/em_R.npy')
rmse = evaluate(em_mu, em_R)
rmse

100%|██████████| 137327/137327 [01:16<00:00, 1806.93it/s]


array([[0.9169972]], dtype=float32)

In [31]:
mcmichael_mu = np.load('results/mcmichael_mu.npy')
mcmichael_R = np.load('results/mcmichael_R.npy')
rmse = evaluate(mcmichael_mu, mcmichael_R)
rmse

100%|██████████| 137327/137327 [01:14<00:00, 1837.93it/s]


array([[0.91700095]], dtype=float32)